# FASE 2: Análisis Relacional - ACTUALIZADO
## Streaming Platform - User-Content Connections & Community Detection

**Objetivo:** Analizar las conexiones entre usuarios y contenido para identificar comunidades y patrones de co-visualización.

**Datos:** Datos reales de la carpeta `analyst`:
- `integrated_data.csv` - Conexiones usuario-contenido con métricas de interés
- `country_aggregated.csv` - Datos agregados por país y criptomoneda
- `trends_enriched.csv` - Patrones de tendencias temporales

**Adaptación:** Criptomonedas → Categorías de contenido de streaming
- Bitcoin → Drama
- Ethereum → Comedy
- BNB → Action
- Solana → Documentary
- Tether → Romance

**Audiencia:** Equipo de algoritmos de recomendación
**Acción:** Personalizar algoritmos por comunidad y optimizar recomendaciones
**Formato:** Presentación técnica (máximo 10 slides)

**NOTA:** Esta versión usa solo Python estándar para máxima compatibilidad.

In [ ]:
# ======================================================================
# CELDA 1: Imports y configuración (solo Python estándar)
# ======================================================================

import csv
import json
import os
import random
import math
from collections import defaultdict, Counter
from datetime import datetime

print("✅ Imports cargados (solo Python estándar)")
print("✅ Listo para análisis relacional con datos reales")

In [ ]:
# ======================================================================
# CELDA 2: Mapeo de datos y configuración
# ======================================================================

# Mapeo de criptomonedas a categorías de contenido
crypto_to_content = {
    'Bitcoin': 'Drama',
    'Ethereum': 'Comedy', 
    'BNB': 'Action',
    'Solana': 'Documentary',
    'Tether': 'Romance'
}

# Mapeo de países a regiones de mercado
country_to_region = {
    'US': 'North America',
    'CN': 'Asia',
    'JP': 'Asia',
    'DE': 'Europe',
    'GB': 'Europe',
    'IN': 'Asia',
    'BR': 'South America',
    'CA': 'North America',
    'AU': 'Oceania',
    'KR': 'Asia'
}

print("Mapeo de datos para análisis relacional:")
print("Criptomonedas → Contenido:")
for crypto, content in crypto_to_content.items():
    print(f"  {crypto} → {content}")
print()
print("Países → Regiones de Mercado:")
for country, region in country_to_region.items():
    print(f"  {country} → {region}")

In [ ]:
# ======================================================================
# CELDA 3: Carga de datos desde la carpeta analyst
# ======================================================================

def load_analyst_data():
    """Cargar datos desde los archivos de la carpeta analyst"""
    
    # 1. Cargar datos integrados para conexiones usuario-contenido
    user_content_data = []
    try:
        with open('analyst/integrated_data.csv', 'r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row['crypto'] in crypto_to_content:
                    user_content_data.append({
                        'user_id': f"{row['country_code']}_{row['country']}",
                        'country': row['country'],
                        'country_code': row['country_code'],
                        'crypto': row['crypto'],
                        'content_category': crypto_to_content[row['crypto']],
                        'interest': float(row['interest']),
                        'region': country_to_region.get(row['country_code'], 'Other'),
                        'close_price': float(row['Close']),
                        'volume': float(row['Volume']),
                        'returns': float(row['returns']),
                        'volatility_7d': float(row['volatility_7d']),
                        'date': row['Date']
                    })
    except Exception as e:
        print(f"Error cargando integrated_data.csv: {e}")
    
    # 2. Cargar datos agregados para análisis de comunidades
    community_data = []
    try:
        with open('analyst/country_aggregated.csv', 'r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row['crypto'] in crypto_to_content:
                    community_data.append({
                        'country_code': row['country_code'],
                        'country_name': row['name'],
                        'crypto': row['crypto'],
                        'content_category': crypto_to_content[row['crypto']],
                        'interest_mean': float(row['interest_mean']),
                        'interest_max': float(row['interest_max']),
                        'volatility_7d_mean': float(row['volatility_7d_mean']),
                        'close_mean': float(row['Close_mean']),
                        'region': country_to_region.get(row['country_code'], 'Other')
                    })
    except Exception as e:
        print(f"Error cargando country_aggregated.csv: {e}")
    
    return user_content_data, community_data

# Cargar datos
print("🔗 CARGANDO DATOS PARA ANÁLISIS RELACIONAL...")
print("="*60)
user_content_data, community_data = load_analyst_data()

if user_content_data and community_data:
    print(f"\n📊 Resumen de datos cargados:")
    print(f"  Conexiones usuario-contenido: {len(user_content_data)} registros")
    print(f"  Datos de comunidades: {len(community_data)} registros")
    print(f"  Usuarios únicos: {len(set(d['user_id'] for d in user_content_data))}")
    print(f"  Categorías de contenido: {sorted(set(d['content_category'] for d in user_content_data))}")
    print(f"  Regiones: {sorted(set(d['region'] for d in user_content_data))}")
else:
    print("❌ Error: No se pudieron cargar los datos")

In [ ]:
# ======================================================================
# CELDA 4: NETWORK GRAPH - Conexiones Usuario-Contenido
# ======================================================================

print("🕸️ NETWORK GRAPH: Conexiones Usuario-Contenido")
print("="*70)

class SimpleGraph:
    def __init__(self):
        self.nodes = {}
        self.edges = []
    
    def add_node(self, node_id, **attributes):
        self.nodes[node_id] = attributes
    
    def add_edge(self, node1, node2, weight=1.0, **attributes):
        self.edges.append((node1, node2, weight, attributes))
    
    def get_neighbors(self, node):
        neighbors = []
        for edge in self.edges:
            if edge[0] == node:
                neighbors.append((edge[1], edge[2]))
            elif edge[1] == node:
                neighbors.append((edge[0], edge[2]))
        return neighbors
    
    def degree_centrality(self, node):
        return len(self.get_neighbors(node)) / max(1, len(self.nodes) - 1)
    
    def density(self):
        n = len(self.nodes)
        if n <= 1:
            return 0
        return len(self.edges) / (n * (n - 1) / 2)

def create_network_graph(user_content_data):
    """Crear grafo de red de conexiones usuario-contenido"""
    
    G = SimpleGraph()
    
    # Agregar nodos de usuarios
    users = set(d['user_id'] for d in user_content_data)
    for user in users:
        user_data = next(d for d in user_content_data if d['user_id'] == user)
        G.add_node(user, type='user', region=user_data['region'], country=user_data['country'])
    
    # Agregar nodos de contenido
    content_categories = set(d['content_category'] for d in user_content_data)
    for content in content_categories:
        G.add_node(content, type='content')
    
    # Agregar aristas basadas en conexiones fuertes
    for data in user_content_data:
        if data['interest'] > 30:  # Umbral de conexión fuerte
            connection_strength = data['interest'] / 100.0
            G.add_edge(data['user_id'], data['content_category'], 
                      weight=connection_strength, 
                      interest=data['interest'],
                      volatility=data['volatility_7d'])
    
    return G

# Crear grafo de red
network_graph = create_network_graph(user_content_data)

print(f"📊 Estadísticas del grafo de red:")
print(f"  Nodos: {len(network_graph.nodes)}")
print(f"  Aristas: {len(network_graph.edges)}")
print(f"  Densidad: {network_graph.density():.3f}")

# Calcular centralidad de contenido
print("\n📈 Contenido más influyente (por centralidad de grado):")
content_centrality = {}
content_categories = set(d['content_category'] for d in user_content_data)
for content in content_categories:
    centrality = network_graph.degree_centrality(content)
    content_centrality[content] = centrality
    print(f"  {content}: {centrality:.3f}")

print(f"\n✅ Grafo de red creado con {len(network_graph.nodes)} nodos y {len(network_graph.edges)} aristas")

In [ ]:
# ======================================================================
# CELDA 5: COMMUNITY DETECTION - Clusters de Usuarios
# ======================================================================

print("👥 COMMUNITY DETECTION: Clusters de Usuarios")
print("="*70)

def detect_communities(user_content_data):
    """Detectar comunidades basadas en preferencias de contenido"""
    
    communities = defaultdict(list)
    user_preferences = defaultdict(list)
    
    # Agrupar preferencias por usuario
    for data in user_content_data:
        if data['interest'] > 25:  # Umbral de preferencia
            user_preferences[data['user_id']].append({
                'content': data['content_category'],
                'interest': data['interest'],
                'region': data['region'],
                'volatility': data['volatility_7d']
            })
    
    # Crear comunidades basadas en patrones de preferencia
    community_id = 0
    for user, preferences in user_preferences.items():
        if len(preferences) > 0:
            # Ordenar preferencias por interés
            preferences.sort(key=lambda x: x['interest'], reverse=True)
            
            # Crear clave de comunidad basada en top 2 preferencias
            top_preferences = [p['content'] for p in preferences[:2]]
            pref_key = tuple(sorted(top_preferences))
            
            if pref_key not in communities:
                communities[pref_key] = {
                    'id': community_id,
                    'users': [],
                    'preferred_content': list(pref_key),
                    'size': 0,
                    'avg_interest': 0,
                    'regions': defaultdict(int),
                    'avg_volatility': 0
                }
                community_id += 1
            
            communities[pref_key]['users'].append(user)
            communities[pref_key]['size'] += 1
            
            # Agregar información de región
            user_region = preferences[0]['region']
            communities[pref_key]['regions'][user_region] += 1
            
            # Calcular interés promedio
            total_interest = sum(p['interest'] for p in preferences)
            communities[pref_key]['avg_interest'] += total_interest
            
            # Calcular volatilidad promedio
            total_volatility = sum(p['volatility'] for p in preferences)
            communities[pref_key]['avg_volatility'] += total_volatility
    
    # Calcular promedios finales
    for community in communities.values():
        if community['size'] > 0:
            community['avg_interest'] /= community['size']
            community['avg_volatility'] /= community['size']
    
    return list(communities.values())

# Detectar comunidades
communities = detect_communities(user_content_data)

print(f"✅ Detectadas {len(communities)} comunidades")

print("\n📊 Análisis de Comunidades:")
for community in communities:
    print(f"\n🔸 Comunidad {community['id']}:")
    print(f"  Tamaño: {community['size']} usuarios")
    print(f"  Contenido preferido: {community['preferred_content']}")
    print(f"  Interés promedio: {community['avg_interest']:.1f}")
    print(f"  Volatilidad promedio: {community['avg_volatility']:.3f}")
    
    # Mostrar distribución por región
    print(f"  Distribución por región:")
    for region, count in community['regions'].items():
        percentage = (count / community['size']) * 100
        print(f"    {region}: {count} usuarios ({percentage:.1f}%)")

print(f"\n✅ Análisis de comunidades completado")

In [ ]:
# ======================================================================
# CELDA 6: FORCE-DIRECTED GRAPH - Visualización de Conexiones
# ======================================================================

print("⚡ FORCE-DIRECTED GRAPH: Visualización de Conexiones")
print("="*70)

def create_force_directed_data(network_graph):
    """Crear datos para gráfico de fuerza dirigida"""
    
    # Simular posiciones basadas en centralidad y conexiones
    node_positions = {}
    
    # Posicionar nodos de contenido en el centro
    content_nodes = [node for node, attrs in network_graph.nodes.items() if attrs.get('type') == 'content']
    user_nodes = [node for node, attrs in network_graph.nodes.items() if attrs.get('type') == 'user']
    
    # Posicionar contenido en círculo central
    for i, content in enumerate(content_nodes):
        angle = (2 * math.pi * i) / len(content_nodes)
        node_positions[content] = {
            'x': 0.5 + 0.2 * math.cos(angle),
            'y': 0.5 + 0.2 * math.sin(angle),
            'type': 'content',
            'centrality': network_graph.degree_centrality(content)
        }
    
    # Posicionar usuarios alrededor del contenido
    for i, user in enumerate(user_nodes):
        # Obtener contenido preferido del usuario
        neighbors = network_graph.get_neighbors(user)
        if neighbors:
            # Posicionar cerca del contenido más conectado
            preferred_content = max(neighbors, key=lambda x: x[1])[0]
            base_pos = node_positions[preferred_content]
            
            # Agregar variación aleatoria
            angle = random.uniform(0, 2 * math.pi)
            distance = random.uniform(0.3, 0.5)
            
            node_positions[user] = {
                'x': base_pos['x'] + distance * math.cos(angle),
                'y': base_pos['y'] + distance * math.sin(angle),
                'type': 'user',
                'region': network_graph.nodes[user].get('region', 'Other'),
                'centrality': network_graph.degree_centrality(user)
            }
        else:
            # Posición aleatoria si no hay conexiones
            node_positions[user] = {
                'x': random.uniform(0, 1),
                'y': random.uniform(0, 1),
                'type': 'user',
                'region': network_graph.nodes[user].get('region', 'Other'),
                'centrality': 0
            }
    
    return node_positions

# Crear datos de fuerza dirigida
force_directed_data = create_force_directed_data(network_graph)

print(f"📊 Datos de fuerza dirigida creados:")
print(f"  Nodos posicionados: {len(force_directed_data)}")

# Análisis de clusters visuales
content_clusters = defaultdict(list)
for node, pos in force_directed_data.items():
    if pos['type'] == 'user':
        # Encontrar contenido más cercano
        min_distance = float('inf')
        closest_content = None
        
        for content_node, content_pos in force_directed_data.items():
            if content_pos['type'] == 'content':
                distance = math.sqrt((pos['x'] - content_pos['x'])**2 + (pos['y'] - content_pos['y'])**2)
                if distance < min_distance:
                    min_distance = distance
                    closest_content = content_node
        
        if closest_content:
            content_clusters[closest_content].append(node)

print("\n📊 Clusters visuales por contenido:")
for content, users in content_clusters.items():
    print(f"  {content}: {len(users)} usuarios cercanos")

print(f"\n✅ Datos de fuerza dirigida preparados")

In [ ]:
# ======================================================================
# CELDA 7: ADJACENCY MATRIX HEATMAP - Matriz de Conexiones
# ======================================================================

print("🔥 ADJACENCY MATRIX HEATMAP: Matriz de Conexiones")
print("="*70)

def create_adjacency_matrix(user_content_data):
    """Crear matriz de adyacencia usuario-contenido"""
    
    # Obtener usuarios y contenido únicos
    users = sorted(set(d['user_id'] for d in user_content_data))
    content_categories = sorted(set(d['content_category'] for d in user_content_data))
    
    # Crear matriz de conexiones
    adjacency_matrix = {}
    for user in users:
        adjacency_matrix[user] = {}
        for content in content_categories:
            adjacency_matrix[user][content] = 0
    
    # Llenar matriz con datos reales
    for data in user_content_data:
        user = data['user_id']
        content = data['content_category']
        interest = data['interest']
        
        # Normalizar interés a escala 0-1
        normalized_interest = min(interest / 100.0, 1.0)
        adjacency_matrix[user][content] = normalized_interest
    
    return adjacency_matrix, users, content_categories

# Crear matriz de adyacencia
adjacency_matrix, users, content_categories = create_adjacency_matrix(user_content_data)

print(f"📊 Matriz de adyacencia creada:")
print(f"  Usuarios: {len(users)}")
print(f"  Categorías de contenido: {len(content_categories)}")
print(f"  Dimensiones: {len(users)} x {len(content_categories)}")

# Análisis de la matriz
print("\n📊 Análisis de conexiones por usuario:")
user_connection_strengths = {}
for user in users[:10]:  # Mostrar solo los primeros 10 usuarios
    total_connection = sum(adjacency_matrix[user].values())
    user_connection_strengths[user] = total_connection
    print(f"  {user}: {total_connection:.2f} (conexión total)")

print("\n📊 Análisis de conexiones por contenido:")
content_connection_strengths = {}
for content in content_categories:
    total_connection = sum(adjacency_matrix[user][content] for user in users)
    content_connection_strengths[content] = total_connection
    print(f"  {content}: {total_connection:.2f} (conexión total)")

print(f"\n✅ Matriz de adyacencia preparada")

In [ ]:
# ======================================================================
# CELDA 8: CHORD DIAGRAM - Patrones de Co-visualización
# ======================================================================

print("🎵 CHORD DIAGRAM: Patrones de Co-visualización")
print("="*70)

def create_chord_diagram_data(user_content_data):
    """Crear datos para diagrama de cuerdas de co-visualización"""
    
    # Agrupar usuarios por contenido
    content_users = defaultdict(set)
    for data in user_content_data:
        if data['interest'] > 20:  # Umbral de interés
            content_users[data['content_category']].add(data['user_id'])
    
    # Calcular similitudes entre contenidos
    similarities = {}
    content_list = list(content_users.keys())
    
    for i, content1 in enumerate(content_list):
        for j, content2 in enumerate(content_list):
            if i != j:
                users1 = content_users[content1]
                users2 = content_users[content2]
                
                # Calcular similitud de Jaccard
                intersection = len(users1.intersection(users2))
                union = len(users1.union(users2))
                similarity = intersection / union if union > 0 else 0
                
                similarities[(content1, content2)] = similarity
    
    return similarities, content_users

# Crear datos del diagrama de cuerdas
chord_similarities, content_users = create_chord_diagram_data(user_content_data)

print(f"📊 Patrones de co-visualización analizados:")
print(f"  Categorías de contenido: {len(content_users)}")
print(f"  Similitudes calculadas: {len(chord_similarities)}")

print("\n🔸 Pares con mayor similitud de audiencia:")
sorted_similarities = sorted(chord_similarities.items(), key=lambda x: x[1], reverse=True)
for (content1, content2), similarity in sorted_similarities:
    if similarity > 0.1:  # Solo mostrar similitudes significativas
        print(f"  {content1} ↔ {content2}: {similarity:.3f}")

print("\n📊 Audiencia por categoría de contenido:")
for content, users in content_users.items():
    print(f"  {content}: {len(users)} usuarios únicos")

print(f"\n✅ Datos del diagrama de cuerdas preparados")

In [ ]:
# ======================================================================
# CELDA 9: GRAPH METRICS DASHBOARD - Métricas de Red
# ======================================================================

print("📊 GRAPH METRICS DASHBOARD: Métricas de Red")
print("="*70)

def calculate_graph_metrics(network_graph, user_content_data):
    """Calcular métricas de red"""
    
    metrics = {
        'total_nodes': len(network_graph.nodes),
        'total_edges': len(network_graph.edges),
        'density': network_graph.density(),
        'user_nodes': len([n for n, attrs in network_graph.nodes.items() if attrs.get('type') == 'user']),
        'content_nodes': len([n for n, attrs in network_graph.nodes.items() if attrs.get('type') == 'content']),
        'avg_connections_per_user': 0,
        'avg_connections_per_content': 0,
        'strong_connections': 0,
        'weak_connections': 0
    }
    
    # Calcular conexiones promedio
    user_connections = 0
    content_connections = 0
    
    for node, attrs in network_graph.nodes.items():
        neighbors = network_graph.get_neighbors(node)
        if attrs.get('type') == 'user':
            user_connections += len(neighbors)
        elif attrs.get('type') == 'content':
            content_connections += len(neighbors)
    
    if metrics['user_nodes'] > 0:
        metrics['avg_connections_per_user'] = user_connections / metrics['user_nodes']
    
    if metrics['content_nodes'] > 0:
        metrics['avg_connections_per_content'] = content_connections / metrics['content_nodes']
    
    # Clasificar conexiones por fuerza
    for edge in network_graph.edges:
        weight = edge[2]
        if weight > 0.5:
            metrics['strong_connections'] += 1
        else:
            metrics['weak_connections'] += 1
    
    return metrics

# Calcular métricas
graph_metrics = calculate_graph_metrics(network_graph, user_content_data)

print(f"📊 Métricas de la red:")
print(f"  Nodos totales: {graph_metrics['total_nodes']}")
print(f"  Aristas totales: {graph_metrics['total_edges']}")
print(f"  Densidad: {graph_metrics['density']:.3f}")
print(f"  Nodos de usuario: {graph_metrics['user_nodes']}")
print(f"  Nodos de contenido: {graph_metrics['content_nodes']}")
print(f"  Conexiones promedio por usuario: {graph_metrics['avg_connections_per_user']:.2f}")
print(f"  Conexiones promedio por contenido: {graph_metrics['avg_connections_per_content']:.2f}")
print(f"  Conexiones fuertes: {graph_metrics['strong_connections']}")
print(f"  Conexiones débiles: {graph_metrics['weak_connections']}")

# Análisis de centralidad
print("\n📈 Análisis de centralidad:")
centrality_scores = {}
for node, attrs in network_graph.nodes.items():
    centrality = network_graph.degree_centrality(node)
    centrality_scores[node] = centrality

# Top nodos más centrales
top_central_nodes = sorted(centrality_scores.items(), key=lambda x: x[1], reverse=True)[:10]
print("\n🔸 Top 10 nodos más centrales:")
for node, centrality in top_central_nodes:
    node_type = network_graph.nodes[node].get('type', 'unknown')
    print(f"  {node} ({node_type}): {centrality:.3f}")

print(f"\n✅ Dashboard de métricas completado")

In [ ]:
# ======================================================================
# CELDA 10: Análisis de Insights y Recomendaciones
# ======================================================================

print("💡 ANÁLISIS DE INSIGHTS Y RECOMENDACIONES")
print("="*70)

# Análisis de la red
print("\n🕸️ INSIGHTS DE LA RED:")

# Contenido más central
most_central_content = max(content_centrality.items(), key=lambda x: x[1])
print(f"• Contenido más central: {most_central_content[0]} (centralidad: {most_central_content[1]:.3f})")

# Comunidad más grande
largest_community = max(communities, key=lambda x: x['size'])
print(f"• Comunidad más grande: {largest_community['id']} ({largest_community['size']} usuarios)")
print(f"  - Contenido preferido: {largest_community['preferred_content']}")
print(f"  - Interés promedio: {largest_community['avg_interest']:.1f}")

# Densidad de la red
print(f"• Densidad de la red: {graph_metrics['density']:.3f}")
if graph_metrics['density'] > 0.1:
    print("  - Red bien conectada, buenas oportunidades de recomendación cruzada")
else:
    print("  - Red dispersa, oportunidades de conectar usuarios similares")

# Análisis de co-visualización
print("\n🎵 INSIGHTS DE CO-VISUALIZACIÓN:")

# Mayor similitud de audiencia
if chord_similarities:
    strongest_similarity = max(chord_similarities.items(), key=lambda x: x[1])
    print(f"• Mayor similitud de audiencia: {strongest_similarity[0][0]} ↔ {strongest_similarity[0][1]} ({strongest_similarity[1]:.3f})")
    
    # Recomendaciones basadas en similitud
    if strongest_similarity[1] > 0.3:
        print(f"  - Oportunidad de recomendación cruzada entre {strongest_similarity[0][0]} y {strongest_similarity[0][1]}")

# Análisis de comunidades
print("\n👥 INSIGHTS DE COMUNIDADES:")

# Comunidad más diversa geográficamente
most_diverse_community = max(communities, key=lambda x: len(x['regions']))
print(f"• Comunidad más diversa: {most_diverse_community['id']} ({len(most_diverse_community['regions'])} regiones)")

# Comunidad con mayor volatilidad (tendencias)
most_volatile_community = max(communities, key=lambda x: x['avg_volatility'])
print(f"• Comunidad más volátil: {most_volatile_community['id']} (volatilidad: {most_volatile_community['avg_volatility']:.3f})")

# Recomendaciones
print("\n🎯 RECOMENDACIONES:")
print(f"\n1. ALGORITMOS DE RECOMENDACIÓN:")
print(f"   • Personalizar por comunidad: {largest_community['preferred_content']}")
print(f"   • Implementar recomendaciones cruzadas para {most_central_content[0]}")
print(f"   • Usar similitud de audiencia para {strongest_similarity[0][0]} ↔ {strongest_similarity[0][1]}")

print(f"\n2. ESTRATEGIA DE CONTENIDO:")
print(f"   • Priorizar {most_central_content[0]} en algoritmos de recomendación")
print(f"   • Desarrollar contenido puente entre {strongest_similarity[0][0]} y {strongest_similarity[0][1]}")
print(f"   • Monitorear tendencias en {most_volatile_community['preferred_content']}")

print(f"\n3. EXPANSIÓN DE AUDIENCIA:")
print(f"   • Enfocar en {most_diverse_community['preferred_content']} para expansión global")
print(f"   • Crear campañas dirigidas a comunidades de {largest_community['preferred_content']}")
print(f"   • Desarrollar contenido híbrido para conectar comunidades")

# Guardar resultados
results = {
    'network_insights': {
        'most_central_content': most_central_content[0],
        'largest_community': largest_community['id'],
        'network_density': graph_metrics['density'],
        'strongest_similarity': strongest_similarity[0] if chord_similarities else None
    },
    'community_insights': {
        'most_diverse_community': most_diverse_community['id'],
        'most_volatile_community': most_volatile_community['id'],
        'total_communities': len(communities)
    },
    'raw_data': {
        'network_graph': {
            'nodes': len(network_graph.nodes),
            'edges': len(network_graph.edges),
            'density': graph_metrics['density']
        },
        'communities': [{
            'id': c['id'],
            'size': c['size'],
            'preferred_content': c['preferred_content'],
            'avg_interest': c['avg_interest'],
            'regions': dict(c['regions'])
        } for c in communities],
        'chord_similarities': dict(chord_similarities),
        'content_centrality': content_centrality
    }
}

with open('phase2_relational_results_updated.json', 'w') as f:
    json.dump(results, f, indent=2, default=str)

print("\n✅ Resultados guardados en: phase2_relational_results_updated.json")

In [ ]:
# ======================================================================
# CELDA 11: RESUMEN Y BIG IDEA DE LA FASE 2
# ======================================================================

print("🎯 RESUMEN DE LA FASE 2: ANÁLISIS RELACIONAL (ACTUALIZADO)")
print("="*80)

print("\n📊 ANÁLISIS COMPLETADOS:")
print("✅ 1. Network Graph: Conexiones usuario-contenido")
print("✅ 2. Community Detection: Clusters de usuarios")
print("✅ 3. Force-Directed Graph: Visualización de conexiones")
print("✅ 4. Adjacency Matrix Heatmap: Matriz de conexiones")
print("✅ 5. Chord Diagram: Patrones de co-visualización")
print("✅ 6. Graph Metrics Dashboard: Métricas de red")

print("\n🔍 INSIGHTS PRINCIPALES:")
print(f"• Contenido más central: {most_central_content[0]} (centralidad: {most_central_content[1]:.3f})")
print(f"• Comunidad más grande: {largest_community['id']} ({largest_community['size']} usuarios)")
print(f"• Densidad de red: {graph_metrics['density']:.3f}")
if chord_similarities:
    print(f"• Mayor similitud de audiencia: {strongest_similarity[0][0]} ↔ {strongest_similarity[0][1]} ({strongest_similarity[1]:.3f})")
print(f"• Comunidades detectadas: {len(communities)}")

print("\n💡 BIG IDEA:")
print(f"Nuestro análisis relacional revela que los usuarios forman comunidades")
print(f"distintas basadas en preferencias de contenido, con {most_central_content[0]} como")
print(f"contenido más central. Debemos personalizar algoritmos de recomendación")
print(f"por comunidad y aprovechar contenido puente para expandir audiencias")
print(f"y maximizar el engagement.")

print("\n📁 ARCHIVOS GENERADOS:")
print("• phase2_relational_results_updated.json")

print("\n" + "="*80)
print("FASE 2 COMPLETADA EXITOSAMENTE (ACTUALIZADA CON DATOS REALES)")
print("="*80)